In [ ]:
%pip install transformers datasets evaluate

In [3]:
from typing import List
import os
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModelForMultipleChoice
import torch
from torch import nn

In [2]:
# data paths 
DIR = os.getcwd()
TRAIN_FEATURES_FILE = "train_features_X4juyT6.csv"
TRAIN_LABELS_FILE = "train_labels_JxtENGl.csv"
TEST_FEATURES_FILE = "smoke_test_features_bWOfr2M.csv"
TEST_LABELS_FILE = "smoke_test_labels_waBGl8d.csv"

TRAIN_FEATURES_PATH = DIR + f"/data/{TRAIN_FEATURES_FILE}"
TRAIN_LABELS_PATH = DIR + f"/data/{TRAIN_LABELS_FILE}"
TEST_FEATURES_PATH = DIR + f"/data/{TEST_FEATURES_FILE}"
TEST_LABELS_PATH = DIR + f"/data/{TEST_LABELS_FILE}"

In [3]:
# load data 
train_features_df = pd.read_csv(TRAIN_FEATURES_PATH)
train_labels_df = pd.read_csv(TRAIN_LABELS_PATH)

In [4]:
# load RoBERTa
model_checkpoint = "Payoto/roberta-base-finetuned-swag"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

/Users/edwardyao/anaconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
def answer_depressed(prompt, tokenizer, model):
    
    candidate1 = "not depressed"
    candidate2 = "depressed"
    
    inputs = tokenizer([[prompt, candidate1], [prompt, candidate2]], return_tensors="pt", padding=True)
    labels = torch.tensor(0).unsqueeze(0)
    
    outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()}, labels=labels)
    logits = outputs.logits

    answer = logits.argmax().item()
    
    return answer

In [7]:
llm_answers_depressed = {}
for idx in range(len(train_features_df)):
    
    p = train_features_df['NarrativeCME'][idx]
    try:
        answer = answer_depressed(p, tokenizer, model)
        llm_answers_depressed[idx] = answer
    except:
        print(len(p))

Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors


2432
2618
3923
2282
2430
2831
2672
2610
5407
2535
2924
2352
3804
3663
2315
2959
2676
2694
3101
2579
2578
2389
2342
2568
3135
3285
2620
3671
2288
2507
3570
3636
2663
2548
2256
2285
2165
3213
3113
2583
2878
2772
2432
3227
2472
2863
2485
3484
2870
3448
2499
2453
2951
3405
3056
3950
2356
2356
3448
2384
2821
2566
3427
2470
2499
2246
2481
3175
2724
2844
3415
3216
2872
2907
2785
3605
2961
7017
3815
3370
2338
3847
3328
2528
3011
3095
2498
2270
4123


In [12]:
actual_answer = []
predicted_answer = []

correct = []

for k, v in llm_answers_depressed.items():
    
    actual = train_labels_df['DepressedMood'][k]
    predicted = v
    actual_answer.append(actual)
    predicted_answer.append(v)
    
    correct.append(actual == predicted)

sum(correct)

1287

In [13]:
sum(correct) / len(correct)

0.3290718486320634

In [6]:
class FCN(nn.Module):
    
    def __init__(self, 
                 in_features: int,
                 out_dims: List[int],
                 task_classes: List[int],
                 dropout: float):
        super().__init__()
        
        layers = []
        input_size = in_features
        for dim in out_dims:
            layers.append(nn.Linear(input_size, dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            input_size = dim
        
        self.base_model = nn.Sequential(*layers)
        self.classification_layers = []
        for task in task_classes:
            if task > 2: 
                self.classification_layers.append(nn.Sequential(nn.Linear(input_size, task),
                                                          nn.Softmax(dim=1)))
            else:
                self.classification_layers.append(nn.Sequential(nn.Linear(input_size, task),
                                                          nn.Sigmoid()))
    def forward(self, x: torch.tensor) -> List[torch.tensor]:
        
        out = self.base_model(x)
        output_labels = []
        for class_head in self.classification_layers:
            output_labels.append(class_head(out))
        
        return output_labels

In [7]:
net = FCN(in_features=2, out_dims=[256,256], task_classes=[2,2,2],dropout=0.5)

In [13]:
for layer in net.classification_layers:
    print(len(layer.parameters()))

TypeError: object of type 'generator' has no len()